In [1]:
import os,sys
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import gc
import cv2
import tqdm
import random

In [2]:
#File Load
pipline ="\\"
#if linux piplin="/"
basepath = "DataSet(190408)/"
path = [os.path.join(basepath, x) for x in os.listdir(basepath)] 
path2 = [os.path.join(x, y) for x in path if os.path.isdir(x) for y in os.listdir(x)]
path3= [os.path.join(x, y) for x in path2 if x[-6:] =="_Truth" for y in os.listdir(x)]
path2 = [os.path.join(x, y) for x in path3 if os.path.isdir(x) for y in os.listdir(x)]
path3 = [os.path.join(x, y) for x in path2 if os.path.isdir(x) for y in os.listdir(x) if y[-3:] =="txt"]
FilePath = [[x.replace("Ground_Truth", "Parse_Data").replace("txt","jpg"), x ]for x in path3 ]

In [3]:
def find_ball(label):
    
    ball_position = []
    f = open(label,"r")
    while True:
        line = f.readline()
        if not line: break
        if line.split(",")[0].split("\'")[1] =='sports ball':
            try:
                ball_position.append([int(x) for x in line.split(",")[1:5]])
            except ValueError:
                ball_position.append([int(x) for x in line.split(",")[1:4]] + [int(line.split(",")[4][:-2])])
                
    return ball_position

In [4]:
def all_box_checker(box1, n_x,n_y,n_w,n_h, in_out):
    s_x, s_y, s_w, s_h = box1
    
    bcx = (s_x + s_x + s_w)/2
    bcy = (s_y+s_y+s_h)/2
    if in_out == "in":
        
        if n_x<=s_x and n_y <=s_y and  n_w >= bcx and n_h >= bcy:
            
            return True
        else : return False
    else:
        
        if n_x> s_x + s_w or n_x + n_w < s_x or n_y > s_y + s_h or n_y + n_h < s_y:
            
            return True
        else : return False

In [5]:
def make_random_box(sport_box,h,w, sport=False):

        
    if len(sport_box) !=0:sport=True
    sport_label =[]
    nsport_label =[]
    print("Sports ball? : ", sport)
    
    while True:
        
        n_x = random.randrange(0, w)
        n_y = random.randrange(0, h)
        #This is all random position for n_x and n_y
#         n_w = random.randrange(n_x, w)
#         n_h = random.randrange(n_y, h)
    
        #스포츠볼이 있다면
        if sport == True:
            for box in sport_box:
                #print(box)
                n_w = n_x + box[2]
                n_h = n_y + box[3]
                check = all_box_checker(box, n_x,n_y,n_w,n_h, "in")
                #print(check)
                if check == True: sport_label.append([n_x, n_y, n_w, n_h])
                
            if len(sport_label) > 4:
                sport = False
        
    
        #스포츠볼이 없다면
        if not len(nsport_label) >15:
            if len(sport_box) == 0:
                n_w = n_x + 10
                n_h = n_y + 10
                nsport_label.append([n_x, n_y, n_w, n_h])
            else:
                for box in sport_box:
                    n_w = n_x + box[2]
                    n_h = n_y + box[3]
                    check2 = all_box_checker(box, n_x,n_y,n_w,n_h, "out")

                if check2 == True:  
                    nsport_label.append([n_x, n_y, n_w, n_h])




        if sport == False and len(nsport_label) >15:
            break
    print("Sport ball : ", len(sport_label), " NSport ball : ", len(nsport_label))
    
    
    
    return sport_label, nsport_label

In [6]:
def np_crop(img,crop):
    return img[crop[1]:crop[3],crop[0]:crop[2],:]

In [7]:
ball_count = 0
nball_count = 0
for i,File in enumerate(FilePath):

    img, label = File[0], File[1]
    print(label)
    ball_posi = find_ball(label)
    img_maps = cv2.imread(img)
    h,w,_ = np.shape(img_maps)
    print("Making RandomBox Start..")
    sport_label, nsport_label = make_random_box(ball_posi,h,w)
    print("RandomBox made done ..")
    #print(nsport_label)
    #print(img_maps)
    folder_path1 = os.path.join(File[1].split(pipline)[0],"box_crop")
    folder_path2 = os.path.join(File[1].split(pipline)[0],"box_crop", File[1].split(pipline)[2])
    folder_path3 = os.path.join(File[1].split(pipline)[0],"box_crop", File[1].split(pipline)[2], File[1].split(pipline)[3])

    if not os.path.exists(folder_path1): os.mkdir(folder_path1)
    if not os.path.exists(folder_path2): os.mkdir(folder_path2)
    if not os.path.exists(folder_path3): os.mkdir(folder_path3)
    if not os.path.exists(os.path.join(folder_path3, "Ball")): os.mkdir(os.path.join(folder_path3, "Ball"))
    if not os.path.exists(os.path.join(folder_path3, "NoBall")): os.mkdir(os.path.join(folder_path3, "NoBall"))
        
    print("Save Strat..")
    for a,n in enumerate(ball_posi):
        n[2] = n[0] + n[2]
        n[3] = n[1] + n[3]
        crop = np_crop(img_maps, n)
        cv2.imwrite(os.path.join(folder_path3,"Ball",str(i)+ str(a) + "ori"+ "ball.jpg"), np.array(crop, dtype="uint8"))
    print("Ori.. Done..")
    
    ball_count += len(ball_posi)    


    for n, posi in enumerate(sport_label):
        crop = np_crop(img_maps, posi)
        cv2.imwrite(os.path.join(folder_path3,"Ball",str(i)+str(n)+ "ball.jpg"), np.array(crop, dtype="uint8"))
    print("Posiball..  Done..")
    ball_count += len(sport_label)

    for n, posi in enumerate(nsport_label):
        crop2 =  np_crop(img_maps, posi)
        cv2.imwrite(os.path.join(folder_path3,"NoBall",str(i)+str(n)+ "noball.jpg"), np.array(crop2, dtype="uint8"))
    print("Negaball..  Done..")
    nball_count += len(nsport_label)
    
    gc.collect()


DataSet(190408)/Free_A\Ground_Truth\00000\0\F1.txt
Making RandomBox Start..
Sports ball? :  False
Sport ball :  0  NSport ball :  16
RandomBox made done ..
Save Strat..
Ori.. Done..
Posiball..  Done..
Negaball..  Done..
DataSet(190408)/Free_A\Ground_Truth\00000\0\F2.txt
Making RandomBox Start..
Sports ball? :  False
Sport ball :  0  NSport ball :  16
RandomBox made done ..
Save Strat..
Ori.. Done..
Posiball..  Done..
Negaball..  Done..
DataSet(190408)/Free_A\Ground_Truth\00000\0\F3.txt
Making RandomBox Start..
Sports ball? :  False
Sport ball :  0  NSport ball :  16
RandomBox made done ..
Save Strat..
Ori.. Done..
Posiball..  Done..
Negaball..  Done..
DataSet(190408)/Free_A\Ground_Truth\00000\1\1_10.txt
Making RandomBox Start..
Sports ball? :  False
Sport ball :  0  NSport ball :  16
RandomBox made done ..
Save Strat..
Ori.. Done..
Posiball..  Done..
Negaball..  Done..
DataSet(190408)/Free_A\Ground_Truth\00000\1\1_11.txt
Making RandomBox Start..
Sports ball? :  False
Sport ball :  0  N

DataSet(190408)/Free_A\Ground_Truth\00002\2\3_26.txt
Making RandomBox Start..
Sports ball? :  True
Sport ball :  5  NSport ball :  16
RandomBox made done ..
Save Strat..
Ori.. Done..
Posiball..  Done..
Negaball..  Done..
DataSet(190408)/Free_A\Ground_Truth\00002\2\3_27.txt
Making RandomBox Start..
Sports ball? :  True
Sport ball :  5  NSport ball :  16
RandomBox made done ..
Save Strat..
Ori.. Done..
Posiball..  Done..
Negaball..  Done..
DataSet(190408)/Free_A\Ground_Truth\00002\2\3_28.txt
Making RandomBox Start..
Sports ball? :  True
Sport ball :  5  NSport ball :  16
RandomBox made done ..
Save Strat..
Ori.. Done..
Posiball..  Done..
Negaball..  Done..
DataSet(190408)/Free_A\Ground_Truth\00002\3\3_29.txt
Making RandomBox Start..
Sports ball? :  True
Sport ball :  5  NSport ball :  16
RandomBox made done ..
Save Strat..
Ori.. Done..
Posiball..  Done..
Negaball..  Done..
DataSet(190408)/Free_A\Ground_Truth\00002\3\3_30.txt
Making RandomBox Start..
Sports ball? :  True
Sport ball :  5  

DataSet(190408)/Free_A\Ground_Truth\00005\0\Free1 (2).txt
Making RandomBox Start..
Sports ball? :  True
Sport ball :  5  NSport ball :  16
RandomBox made done ..
Save Strat..
Ori.. Done..
Posiball..  Done..
Negaball..  Done..
DataSet(190408)/Free_A\Ground_Truth\00005\0\Free1 (3).txt
Making RandomBox Start..
Sports ball? :  True
Sport ball :  5  NSport ball :  16
RandomBox made done ..
Save Strat..
Ori.. Done..
Posiball..  Done..
Negaball..  Done..
DataSet(190408)/Free_A\Ground_Truth\00005\0\Free1 (5).txt
Making RandomBox Start..
Sports ball? :  True
Sport ball :  5  NSport ball :  16
RandomBox made done ..
Save Strat..
Ori.. Done..
Posiball..  Done..
Negaball..  Done..
DataSet(190408)/Free_A\Ground_Truth\00005\1\Free1 (6).txt
Making RandomBox Start..
Sports ball? :  True
Sport ball :  5  NSport ball :  16
RandomBox made done ..
Save Strat..
Ori.. Done..
Posiball..  Done..
Negaball..  Done..
DataSet(190408)/Free_A\Ground_Truth\00005\1\Free1 (7).txt
Making RandomBox Start..
Sports ball? 

DataSet(190408)/Free_A\Ground_Truth\00007\0\Free1 (81).txt
Making RandomBox Start..
Sports ball? :  True
Sport ball :  5  NSport ball :  16
RandomBox made done ..
Save Strat..
Ori.. Done..
Posiball..  Done..
Negaball..  Done..
DataSet(190408)/Free_A\Ground_Truth\00007\0\Free1 (82).txt
Making RandomBox Start..
Sports ball? :  True
Sport ball :  5  NSport ball :  16
RandomBox made done ..
Save Strat..
Ori.. Done..
Posiball..  Done..
Negaball..  Done..
DataSet(190408)/Free_A\Ground_Truth\00007\0\Free1 (84).txt
Making RandomBox Start..
Sports ball? :  True
Sport ball :  5  NSport ball :  16
RandomBox made done ..
Save Strat..
Ori.. Done..
Posiball..  Done..
Negaball..  Done..
DataSet(190408)/Free_A\Ground_Truth\00007\1\Free1 (85).txt
Making RandomBox Start..
Sports ball? :  True
Sport ball :  5  NSport ball :  16
RandomBox made done ..
Save Strat..
Ori.. Done..
Posiball..  Done..
Negaball..  Done..
DataSet(190408)/Free_A\Ground_Truth\00007\1\Free1 (86).txt
Making RandomBox Start..
Sports b

Negaball..  Done..
DataSet(190408)/Free_A\Ground_Truth\00009\2\Free1 (167).txt
Making RandomBox Start..
Sports ball? :  True
Sport ball :  5  NSport ball :  16
RandomBox made done ..
Save Strat..
Ori.. Done..
Posiball..  Done..
Negaball..  Done..
DataSet(190408)/Free_A\Ground_Truth\00009\2\Free1 (168).txt
Making RandomBox Start..
Sports ball? :  True
Sport ball :  5  NSport ball :  16
RandomBox made done ..
Save Strat..
Ori.. Done..
Posiball..  Done..
Negaball..  Done..
DataSet(190408)/Free_A\Ground_Truth\00009\2\Free1 (169).txt
Making RandomBox Start..
Sports ball? :  True
Sport ball :  5  NSport ball :  16
RandomBox made done ..
Save Strat..
Ori.. Done..
Posiball..  Done..
Negaball..  Done..
DataSet(190408)/Free_A\Ground_Truth\00009\2\Free1 (170).txt
Making RandomBox Start..
Sports ball? :  True
Sport ball :  5  NSport ball :  16
RandomBox made done ..
Save Strat..
Ori.. Done..
Posiball..  Done..
Negaball..  Done..
DataSet(190408)/Free_A\Ground_Truth\00009\3\Free1 (171).txt
Making Ra

...

RandomBox made done ..
Save Strat..
Ori.. Done..
Posiball..  Done..
Negaball..  Done..
DataSet(190408)/Penalty_B\Ground_Truth\50094\1\FIFA 16 - Best Penalty Shootout 4004.txt
Making RandomBox Start..
Sports ball? :  True
Sport ball :  5  NSport ball :  16
RandomBox made done ..
Save Strat..
Ori.. Done..
Posiball..  Done..
Negaball..  Done..
DataSet(190408)/Penalty_B\Ground_Truth\50094\1\FIFA 16 - Best Penalty Shootout 4006.txt
Making RandomBox Start..
Sports ball? :  True
Sport ball :  5  NSport ball :  16
RandomBox made done ..
Save Strat..
Ori.. Done..
Posiball..  Done..
Negaball..  Done..
DataSet(190408)/Penalty_B\Ground_Truth\50094\1\FIFA 16 - Best Penalty Shootout 4009.txt
Making RandomBox Start..
Sports ball? :  True
Sport ball :  5  NSport ball :  16
RandomBox made done ..
Save Strat..
Ori.. Done..
Posiball..  Done..
Negaball..  Done..
DataSet(190408)/Penalty_B\Ground_Truth\50094\2\FIFA 16 - Best Penalty Shootout 4010.txt
Making RandomBox Start..
Sports ball? :  True
Sport ball 

RandomBox made done ..
Save Strat..
Ori.. Done..
Posiball..  Done..
Negaball..  Done..
DataSet(190408)/Penalty_B\Ground_Truth\50096\3\FIFA 16 - Best Penalty Shootout 5452.txt
Making RandomBox Start..
Sports ball? :  True
Sport ball :  5  NSport ball :  16
RandomBox made done ..
Save Strat..
Ori.. Done..
Posiball..  Done..
Negaball..  Done..
DataSet(190408)/Penalty_B\Ground_Truth\50097\0\FIFA 17 - Penalty Shootout [Barcelona vs Real Madrid] 0312.txt
Making RandomBox Start..
Sports ball? :  True
Sport ball :  5  NSport ball :  16
RandomBox made done ..
Save Strat..
Ori.. Done..
Posiball..  Done..
Negaball..  Done..
DataSet(190408)/Penalty_B\Ground_Truth\50097\0\FIFA 17 - Penalty Shootout [Barcelona vs Real Madrid] 0318.txt
Making RandomBox Start..
Sports ball? :  True
Sport ball :  5  NSport ball :  16
RandomBox made done ..
Save Strat..
Ori.. Done..
Posiball..  Done..
Negaball..  Done..
DataSet(190408)/Penalty_B\Ground_Truth\50097\0\FIFA 17 - Penalty Shootout [Barcelona vs Real Madrid] 

DataSet(190408)/Penalty_B\Ground_Truth\50099\1\FIFA 17 - Penalty Shootout [Barcelona vs Real Madrid] 0774.txt
Making RandomBox Start..
Sports ball? :  True
Sport ball :  5  NSport ball :  16
RandomBox made done ..
Save Strat..
Ori.. Done..
Posiball..  Done..
Negaball..  Done..
DataSet(190408)/Penalty_B\Ground_Truth\50099\1\FIFA 17 - Penalty Shootout [Barcelona vs Real Madrid] 0776.txt
Making RandomBox Start..
Sports ball? :  True
Sport ball :  5  NSport ball :  16
RandomBox made done ..
Save Strat..
Ori.. Done..
Posiball..  Done..
Negaball..  Done..
DataSet(190408)/Penalty_B\Ground_Truth\50099\2\FIFA 17 - Penalty Shootout [Barcelona vs Real Madrid] 0777.txt
Making RandomBox Start..
Sports ball? :  True
Sport ball :  5  NSport ball :  16
RandomBox made done ..
Save Strat..
Ori.. Done..
Posiball..  Done..
Negaball..  Done..
DataSet(190408)/Penalty_B\Ground_Truth\50099\2\FIFA 17 - Penalty Shootout [Barcelona vs Real Madrid] 0784.txt
Making RandomBox Start..
Sports ball? :  True
Sport ball

RandomBox made done ..
Save Strat..
Ori.. Done..
Posiball..  Done..
Negaball..  Done..
DataSet(190408)/Penalty_C\Ground_Truth\50102\0\00000.txt
Making RandomBox Start..
Sports ball? :  True
Sport ball :  5  NSport ball :  16
RandomBox made done ..
Save Strat..
Ori.. Done..
Posiball..  Done..
Negaball..  Done..
DataSet(190408)/Penalty_C\Ground_Truth\50102\0\00005.txt
Making RandomBox Start..
Sports ball? :  True
Sport ball :  5  NSport ball :  16
RandomBox made done ..
Save Strat..
Ori.. Done..
Posiball..  Done..
Negaball..  Done..
DataSet(190408)/Penalty_C\Ground_Truth\50102\0\00012.txt
Making RandomBox Start..
Sports ball? :  True
Sport ball :  5  NSport ball :  16
RandomBox made done ..
Save Strat..
Ori.. Done..
Posiball..  Done..
Negaball..  Done..
DataSet(190408)/Penalty_C\Ground_Truth\50102\1\00013.txt
Making RandomBox Start..
Sports ball? :  True
Sport ball :  5  NSport ball :  16
RandomBox made done ..
Save Strat..
Ori.. Done..
Posiball..  Done..
Negaball..  Done..
DataSet(19040

DataSet(190408)/Penalty_C\Ground_Truth\50105\0\00014.txt
Making RandomBox Start..
Sports ball? :  True
Sport ball :  5  NSport ball :  16
RandomBox made done ..
Save Strat..
Ori.. Done..
Posiball..  Done..
Negaball..  Done..
DataSet(190408)/Penalty_C\Ground_Truth\50105\0\00020.txt
Making RandomBox Start..
Sports ball? :  True
Sport ball :  5  NSport ball :  16
RandomBox made done ..
Save Strat..
Ori.. Done..
Posiball..  Done..
Negaball..  Done..
DataSet(190408)/Penalty_C\Ground_Truth\50105\0\00026.txt
Making RandomBox Start..
Sports ball? :  True
Sport ball :  5  NSport ball :  16
RandomBox made done ..
Save Strat..
Ori.. Done..
Posiball..  Done..
Negaball..  Done..
DataSet(190408)/Penalty_C\Ground_Truth\50105\1\00027.txt
Making RandomBox Start..
Sports ball? :  True
Sport ball :  5  NSport ball :  16
RandomBox made done ..
Save Strat..
Ori.. Done..
Posiball..  Done..
Negaball..  Done..
DataSet(190408)/Penalty_C\Ground_Truth\50105\1\00029.txt
Making RandomBox Start..
Sports ball? :  Tr

In [8]:
print("PreProcessing End.. Ball Cropped : ", ball_count, " None Ball Cropped : ", nball_count)

PreProcessing End.. Ball Cropped :  14683  None Ball Cropped :  44336
